# Task 01 siRNA
这段代码实现了一个用于siRNA数据集的深度学习模型训练框架。

### 1. 依赖库的导入

In [1]:
import os  # 文件操作
import torch  # 深度学习框架
import random  # 随机数生成
import numpy as np  # 数值计算
import pandas as pd  # 数据处理

import torch.nn as nn  # 神经网络模块
import torch.optim as optim  # 优化器模块

from tqdm import tqdm  # 进度条显示
from rich import print  # 美化打印输出
from collections import Counter  # 计数器工具

from torch.utils.data import Dataset, DataLoader  # 数据集和数据加载器
from sklearn.model_selection import train_test_split  # 数据集划分
from sklearn.metrics import precision_score, recall_score, mean_absolute_error  # 模型评估指标

# 这些库包括了文件操作、深度学习、数据处理、模型评估等必要的工具。

In [2]:
# 该函数确保了在使用NumPy、Python内置随机数生成器和PyTorch时，所有的随机数生成都是可控的和可复现的，有助于实验结果的一致性。
def set_random_seed(seed):
    # 设置NumPy的随机种子
    np.random.seed(seed)
    # 设置Python内置的随机数生成器的种子
    random.seed(seed)
    # 设置PyTorch的随机种子
    torch.manual_seed(seed)
    # 设置CUDA的随机种子
    torch.cuda.manual_seed(seed)
    # 设置所有CUDA设备的随机种子
    torch.cuda.manual_seed_all(seed)
    # 确保每次卷积算法选择都是确定的
    torch.backends.cudnn.deterministic = True
    # 关闭CuDNN自动优化功能，确保结果可复现
    torch.backends.cudnn.benchmark = False


### 2. 基因组分词器类
该类用于将基因组序列分割成固定长度的n-gram。

In [3]:
class GenomicTokenizer:
    def __init__(self, ngram=5, stride=2):
        # 初始化分词器，设置n-gram长度和步幅
        self.ngram = ngram
        self.stride = stride
        
    def tokenize(self, t):
        # 将输入序列转换为大写
        t = t.upper()
        
        if self.ngram == 1:
            # 如果n-gram长度为1，直接将序列转换为字符列表
            toks = list(t)
        else:
            # 否则，按照步幅对序列进行n-gram分词
            toks = [t[i:i+self.ngram] for i in range(0, len(t), self.stride) if len(t[i:i+self.ngram]) == self.ngram]
        
        # 如果最后一个分词长度小于n-gram，移除最后一个分词
        if len(toks[-1]) < self.ngram:
            toks = toks[:-1]
        
        # 返回分词结果
        return toks

**代码解读**
1. 类初始化：`__init__` 方法接受两个参数 `ngram` 和 `stride`，用于设置分词器的 n-gram 长度和步幅。
2. 分词方法：`tokenize` 方法将输入的序列转换为大写，并根据 `ngram` 和 `stride` 对序列进行分词。
3. `n-gram` 长度为 1 的处理：如果 `ngram` 为 1，直接将序列转换为字符列表。
4. `n-gram` 长度大于 1 的处理：按步幅进行分词，并确保每个分词的长度等于 `ngram`。
5. 最后一个分词的处理：如果最后一个分词长度小于 `ngram`，将其移除。
6. 返回分词结果：返回处理后的分词结果列表。

### 3. 基因组词汇类

该类用于创建一个词汇表，用于将基因组片段映射为索引。

In [4]:
class GenomicVocab:
    def __init__(self, itos):
        # 初始化词汇表，itos是一个词汇表列表
        self.itos = itos
        # 创建从词汇到索引的映射
        self.stoi = {v: k for k, v in enumerate(self.itos)}
        
    @classmethod
    def create(cls, tokens, max_vocab, min_freq):
        # 创建词汇表类方法
        # 统计每个token出现的频率
        freq = Counter(tokens)
        # 选择出现频率大于等于min_freq的token，并且最多保留max_vocab个token
        itos = ['<pad>'] + [o for o, c in freq.most_common(max_vocab - 1) if c >= min_freq]
        # 返回包含词汇表的类实例
        return cls(itos)


**代码解读**
1. 类初始化：`__init__` 方法接受一个参数 `itos`，它是一个词汇表列表。

    - `itos`：从索引到词汇的映射。
    - `stoi`：从词汇到索引的映射，由 `itos` 列表生成。

2. 类方法 `create`：创建词汇表的类方法，用于生成 `GenomicVocab` 类的实例。

    - 参数：
        - `tokens`：所有token的列表。
        - `max_vocab`：词汇表的最大容量。
        - `min_freq`：词汇在被包含到词汇表中的最低频率。
    - 步骤：
        - 统计 `tokens` 中每个token出现的频率。
        - 按照频率从高到低排序，并选择出现频率大于等于 `min_freq` 的token，最多保留 `max_vocab` 个。
        - 在词汇表中添加一个特殊的 `<pad>` token，用于填充序列。
        - 返回包含生成的 `itos` 列表的 `GenomicVocab` 实例。

### 4. siRNA数据集类

该类用于加载siRNA数据，并将序列数据转换为模型可以处理的格式。

In [19]:
class SiRNADataset(Dataset):
    def __init__(self, df, columns, vocab, tokenizer, max_len, is_test=False):
        # 初始化数据集
        self.df = df  # 数据框
        self.columns = columns  # 包含序列的列名
        self.vocab = vocab  # 词汇表
        self.tokenizer = tokenizer  # 分词器
        self.max_len = max_len  # 最大序列长度
        self.is_test = is_test  # 指示是否是测速集

    def __len__(self):
        # 返回数据集的长度
        return len(self.df)

    def __getitem__(self, idx):
        # 获取数据集中的第idx个样本
        row = self.df.iloc[idx]  # 获取第idx行数据
        
        # 对每一列进行分词和编码
        seqs = [self.tokenize_and_encode(row[col]) for col in self.columns]
        if self.is_test:
            # 仅返回编码后的序列（非测试集模式）
            return seqs
        else:
            # 获取目标值并转换为张量（仅在非测试集模式下）
            target = torch.tensor(row['mRNA_remaining_pct'], dtype=torch.float)
            # 返回编码后的序列和目标值
            return seqs, target

    def tokenize_and_encode(self, seq):
        if ' ' in seq:  # 修改过的序列
            tokens = seq.split()  # 按空格分词
        else:  # 常规序列
            tokens = self.tokenizer.tokenize(seq)  # 使用分词器分词
        
        # 将token转换为索引，未知token使用0（<pad>）
        encoded = [self.vocab.stoi.get(token, 0) for token in tokens]
        # 将序列填充到最大长度
        padded = encoded + [0] * (self.max_len - len(encoded))
        # 返回张量格式的序列
        return torch.tensor(padded[:self.max_len], dtype=torch.long)



**代码解读**
1. 初始化：`__init__` 方法初始化数据集的必要参数：

    - `df`：包含数据的Pandas数据框。
    - `columns`：包含序列的列名列表。
    - `vocab`：词汇表对象，用于将token转换为索引。
    - `tokenizer`：分词器对象，用于将序列分割为token。
    - `max_len`：序列的最大长度，所有序列将被填充或截断到这个长度。
    - `is_train`：布尔值，指示数据集是否用于训练（默认为False）。

2. 获取数据集长度：`__len__` 方法返回数据集的样本数量。

3. 获取样本：__getitem__ 方法获取数据集中指定索引的样本：

    - 获取指定行的数据。
    - 对每个包含序列的列进行分词和编码。
    - 获取目标值（mRNA_remaining_pct），并将其转换为张量。
    - 返回编码后的序列和目标值。

4. 分词和编码：`tokenize_and_encode` 方法对输入序列进行分词和编码：

    - 如果序列中包含空格，按空格分词（表示序列已经被修改）。
    - 否则，使用分词器对序列进行分词。
    - 将分词结果转换为索引，未知`token`用`0（<pad>）`表示。
    - 对序列进行填充，使其长度等于最大长度 `max_len`。
    - 返回张量格式的填充序列。

### 5. siRNA Model

这是一个基于GRU的神经网络模型，用于处理siRNA序列。

In [20]:
class SiRNAModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=200, hidden_dim=256, n_layers=3, dropout=0.5):
        super(SiRNAModel, self).__init__()
        
        # 初始化嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        # 初始化GRU层
        self.gru = nn.GRU(embed_dim, hidden_dim, n_layers, bidirectional=True, batch_first=True, dropout=dropout)
        # 初始化全连接层
        self.fc = nn.Linear(hidden_dim * 4, 1)  # hidden_dim * 4 因为GRU是双向的，有n_layers层
        # 初始化Dropout层
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # 将输入序列传入嵌入层
        embedded = [self.embedding(seq) for seq in x]
        outputs = []
        
        # 对每个嵌入的序列进行处理
        for embed in embedded:
            x, _ = self.gru(embed)  # 传入GRU层
            x = self.dropout(x[:, -1, :])  # 取最后一个隐藏状态，并进行dropout处理
            outputs.append(x)
        
        # 将所有序列的输出拼接起来
        x = torch.cat(outputs, dim=1)
        # 传入全连接层
        x = self.fc(x)
        # 返回结果
        return x.squeeze()
    

**代码解读**
1. 初始化方法 `__init__`：

    - `vocab_size`：词汇表大小，用于嵌入层。
    - `embed_dim`：嵌入维度，嵌入层将词汇映射为 `embed_dim` 维向量。
    - `hidden_dim`：隐藏层维度，GRU的隐藏状态维度。
    - `n_layers`：GRU的层数。
    - `dropout`：`Dropout`层的丢弃率，用于防止过拟合。
    
该方法初始化了模型的各层，包括`嵌入层`、`GRU层`、`全连接层`和`Dropout层`。

2. 前向传播方法 `forward`：

    - 将输入序列传入嵌入层进行词汇嵌入。
    - 对每个嵌入的序列进行`GRU`处理，提取最后一个隐藏状态并进行`Dropout`处理。
    - 将所有处理后的序列输出拼接起来，并传入全连接层。
    - 返回经过全连接层后的结果。

### 6. 评估指标计算函数

该函数用于计算模型的各项评估指标，包括精确度、召回率、F1值和评分。

In [21]:
def calculate_metrics(y_true, y_pred, threshold=30):
    # 计算平均绝对误差
    mae = np.mean(np.abs(y_true - y_pred))

    # 将实际值和预测值转换为二进制分类（低于阈值为1，高于或等于阈值为0）
    y_true_binary = (y_true < threshold).astype(int)
    y_pred_binary = (y_pred < threshold).astype(int)

    # 创建掩码，用于筛选预测值在0和阈值之间的样本
    mask = (y_pred >= 0) & (y_pred <= threshold)
    range_mae = mean_absolute_error(y_true[mask], y_pred[mask]) if mask.sum() > 0 else 100

    # 计算精确度、召回率和F1得分
    precision = precision_score(y_true_binary, y_pred_binary, average='binary')
    recall = recall_score(y_true_binary, y_pred_binary, average='binary')
    f1 = 2 * precision * recall / (precision + recall)

    # 计算综合评分
    score = (1 - mae / 100) * 0.5 + (1 - range_mae / 100) * f1 * 0.5

    return score

**代码解读**
1. 计算平均绝对误差 (MAE)：

    - `mae = np.mean(np.abs(y_true - y_pred))`：计算实际值和预测值之间的`平均绝对误差`。

2. 将实际值和预测值转换为二进制分类：

    - `y_true_binary = (y_true < threshold).astype(int)`：如果`实际值`小于`阈值`，设为`1`，否则设为`0`。
    - `y_pred_binary = (y_pred < threshold).astype(int)`：如果`预测值`小于`阈值`，设为`1`，否则设为`0`。

3. 创建掩码：

    - `mask = (y_pred >= 0) & (y_pred <= threshold)`：筛选`预测值`在`0`和`阈值`之间的样本。
    - `range_mae = mean_absolute_error(y_true[mask], y_pred[mask]) if mask.sum() > 0 else 100`：计算这些样本的`平均绝对误差`，如果`没有`符合条件的样本，设为`100`。

4. 计算精确度、召回率和F1得分：

    - `precision = precision_score(y_true_binary, y_pred_binary, average='binary')`：计算精确度。
    - `recall = recall_score(y_true_binary, y_pred_binary, average='binary')`：计算召回率。
    - `f1 = 2 * precision * recall / (precision + recall)`：计算F1得分。

5. 计算综合评分：

    - `score = (1 - mae / 100) * 0.5 + (1 - range_mae / 100) * f1 * 0.5`：综合MAE、范围内的MAE和F1得分，计算最终评分。

6. 返回评分：

    - `return score`：返回综合评分。

### 7. 模型评估函数

该函数用于在测试集上评估模型性能。

In [22]:
def evaluate_model(model, test_loader, device='cuda'):
    # 设置模型为评估模式
    model.eval()
    predictions = []
    targets = []
    
    # 禁用梯度计算
    with torch.no_grad():
        # 遍历测试数据加载器中的每个批次
        for inputs, target in test_loader:
            # 将输入数据移动到指定设备上
            inputs = [x.to(device) for x in inputs]
            # 获取模型的输出
            outputs = model(inputs)
            # 将预测结果从GPU移到CPU，并转换为numpy数组，添加到predictions列表中
            predictions.extend(outputs.cpu().numpy())
            # 将目标值转换为numpy数组，添加到targets列表中
            targets.extend(target.numpy())

    # 将预测结果和目标值转换为numpy数组
    y_pred = np.array(predictions)
    y_test = np.array(targets)
    
    # 计算评估指标
    score = calculate_metrics(y_test, y_pred)
    # 打印测试得分
    print(f"Test Score: {score:.4f}")

### 8. 模型训练函数

函数用于训练模型，并在每个epoch后评估模型的性能，保存最佳模型。

In [23]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, device='cuda', output_dir: str=""):
    # 将模型移动到指定设备
    model.to(device)
    best_score = -float('inf')  # 初始化最佳得分
    best_model = None  # 初始化最佳模型

    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        train_loss = 0  # 初始化训练损失
        for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs = [x.to(device) for x in inputs]  # 将输入移动到设备
            targets = targets.to(device)  # 将目标值移动到设备
            
            optimizer.zero_grad()  # 清空梯度
            outputs = model(inputs)  # 前向传播
            loss = criterion(outputs, targets)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            
            train_loss += loss.item()  # 累加训练损失
        
        model.eval()  # 设置模型为评估模式
        val_loss = 0  # 初始化验证损失
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = [x.to(device) for x in inputs]  # 将输入移动到设备
                targets = targets.to(device)  # 将目标值移动到设备
                outputs = model(inputs)  # 前向传播
                loss = criterion(outputs, targets)  # 计算损失
                val_loss += loss.item()  # 累加验证损失
                val_preds.extend(outputs.cpu().numpy())  # 收集预测值
                val_targets.extend(targets.cpu().numpy())  # 收集目标值
        
        train_loss /= len(train_loader)  # 计算平均训练损失
        val_loss /= len(val_loader)  # 计算平均验证损失
        
        val_preds = np.array(val_preds)
        val_targets = np.array(val_targets)
        score = calculate_metrics(val_targets, val_preds)  # 计算验证集上的得分
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        print(f'Validation Score: {score:.4f}')

        if score > best_score:
            best_score = score  # 更新最佳得分
            best_model = model.state_dict().copy()  # 更新最佳模型
            torch.save(model.state_dict(), os.path.join(output_dir, "best.pt".format(epoch)))  # 保存最佳模型
            print(f'New best model found with score: {best_score:.4f}')

    return best_model  # 返回最佳模型


### 9. 训练主程序

In [24]:
# 设置参数
bs = 64    # 批次大小
epochs = 50    # 训练的迭代次数
lr = 0.001    # 学习率
seed = 42    # 随机种子
output_dir = "output/models"    # 模型保存路径

# 选择设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 设置随机种子以确保结果可重复
set_random_seed(seed)

# 创建输出目录
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 加载数据
train_data = pd.read_csv('train_data.csv')

# 指定需要处理的列
columns = ['siRNA_antisense_seq', 'modified_siRNA_antisense_seq_list']
# 删除包含空值的行
train_data.dropna(subset=columns + ['mRNA_remaining_pct'], inplace=True)
# 将数据分为训练集和验证集
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# 创建分词器
tokenizer = GenomicTokenizer(ngram=3, stride=3)

# 创建词汇表
all_tokens = []
for col in columns:
    for seq in train_data[col]:
        if ' ' in seq:  # 修改过的序列
            all_tokens.extend(seq.split())
        else:
            all_tokens.extend(tokenizer.tokenize(seq))
vocab = GenomicVocab.create(all_tokens, max_vocab=10000, min_freq=1)

# 找到最大序列长度
max_len = max(max(len(seq.split()) if ' ' in seq else len(tokenizer.tokenize(seq)) 
                    for seq in train_data[col]) for col in columns)

# 创建数据集
train_dataset = SiRNADataset(train_data, columns, vocab, tokenizer, max_len)
val_dataset = SiRNADataset(val_data, columns, vocab, tokenizer, max_len)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=bs)

# 初始化模型
model = SiRNAModel(len(vocab.itos))
criterion = nn.MSELoss()

# 初始化优化器
optimizer = optim.Adam(model.parameters(), lr=lr)

# 训练模型
best_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs, device, output_dir=output_dir)


Epoch 1/50: 100%|██████████| 363/363 [00:05<00:00, 66.18it/s]


Epoch 1/50

Train Loss: 1366.2798, Val Loss: 1182.0657

Learning Rate: 0.001000

Validation Score: 0.3624

...

Epoch 50/50: 100%|██████████| 363/363 [00:05<00:00, 63.95it/s]


Epoch 50/50

Train Loss: 493.8637, Val Loss: 811.5681

Learning Rate: 0.001000

Validation Score: 0.6176

### 10. 测试程序

In [25]:
# 设置输出目录
output_dir = "result"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 加载测试数据
test_data = pd.read_csv('sample_submission.csv')
columns = ['siRNA_antisense_seq', 'modified_siRNA_antisense_seq_list']
test_data.dropna(subset=columns, inplace=True)

# 创建分词器
tokenizer = GenomicTokenizer(ngram=3, stride=3)

# 创建词汇表
all_tokens = []
for col in columns:
    for seq in test_data[col]:
        if ' ' in seq:  # 修改过的序列
            all_tokens.extend(seq.split())
        else:
            all_tokens.extend(tokenizer.tokenize(seq))
            
vocab = GenomicVocab.create(all_tokens, max_vocab=10000, min_freq=1)

# 找到最大序列长度
max_len = max(max(len(seq.split()) if ' ' in seq else len(tokenizer.tokenize(seq)) 
                    for seq in test_data[col]) for col in columns)

# 创建测试数据集
test_dataset = SiRNADataset(test_data, columns, vocab, tokenizer, max_len, is_test=True)

# 创建数据加载器
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型
model = SiRNAModel(len(vocab.itos))
model.load_state_dict(best_model)  # 加载最佳模型权重
model.to(device=device)
model.eval()  # 切换到评估模式，这对于某些模块如Dropout和BatchNorm是必需的

# 进行预测
preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        # import pdb;pdb.set_trace()
        inputs = [x.to(device) for x in inputs]
        outputs = model(inputs)
        preds.extend(outputs.cpu().numpy())

# 将预测结果添加到测试数据中
test_data["mRNA_remaining_pct"] = preds
df = pd.DataFrame(test_data)

# 保存预测结果
output_csv = os.path.join(output_dir, "submission.csv")
print(f"submission.csv 保存在 {output_csv}")
df.to_csv(output_csv, index=False)


100%|██████████| 77/77 [00:00<00:00, 132.33it/s]


submission.csv 保存在 result/submission.csv